<a href="https://colab.research.google.com/github/saransh121/GEN-AI/blob/main/LangChainNews.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q langchain langchain-community huggingface_hub
!pip install -q transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 806.7/806.7 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 17.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 238.5/238.5 kB 22.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.4/55.4 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 6.1 MB/s eta 0:00:00


In [2]:
!pip install -q transformers
!pip install -q sentence-transformers
!pip install -q kaleido
!pip install -q python-multipart
!pip install -q chromadb
!pip install -q langchainhub

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.8/132.8 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.9/79.9 MB 8.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
lida 0.0.10 requires fastapi, which is not installed.
lida 0.0.10 requires python-multipart, which is not installed.
lida 0.0.10 requires uvicorn, which is not installed.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
lida 0.0.10 requires fastapi, which is not installed.
lida 0.0.10 requires uvicorn, which is not installed.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 509.0/509.0 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 22.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92

### Importing the model



In [3]:
from langchain.llms import HuggingFaceHub
import os
from getpass import getpass
os.environ["HUGGINGFACEHUB_API_TOKEN"] = getpass("HF Token:")
import bs4
from langchain_community.document_loaders import WebBaseLoader

HF Token:··········


### Loding the model

In [55]:
llm = HuggingFaceHub(repo_id="mistralai/Mistral-7B-v0.1")

##### load the documents form the web

In [22]:
strainer = bs4.SoupStrainer(class_=("mw-body"))

In [23]:
webLoader = WebBaseLoader(web_path="https://en.wikipedia.org/wiki/India",bs_kwargs={"parse_only": strainer})

In [24]:
loaded_data = webLoader.load()

##### visulize the data

In [25]:
print(len(loaded_data[0].page_content))

202640


#### spliting the data to feed in to the LLM model

In [26]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [27]:
spliter = RecursiveCharacterTextSplitter(    chunk_size=200,
    chunk_overlap=30,add_start_index=True
)

In [28]:
docs = spliter.split_documents(loaded_data)

In [29]:
len(docs)

1358

In [30]:
print(docs[60])

page_content="In the early medieval era, Christianity, Islam, Judaism, and Zoroastrianism became established on India's southern and western coasts.[44]" metadata={'source': 'https://en.wikipedia.org/wiki/India', 'start_index': 8773}


##### store the collected data into vector database using hugging face embbading

In [14]:
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma

In [31]:
vectorDb = Chroma.from_documents(docs,HuggingFaceEmbeddings())

### retriver to retrive the data

In [44]:
retriver = vectorDb.as_retriever(search_type="similarity", search_kwargs={"k": 6})

In [60]:
retriver.invoke("where  is Himalayas?")

[Document(page_content='Himalayas.[173] Immediately south of the emerging Himalayas, plate movement created a vast crescent-shaped trough that rapidly filled with river-borne sediment[174] and now constitutes the', metadata={'source': 'https://en.wikipedia.org/wiki/India', 'start_index': 29013}),
 Document(page_content='the Himalayan foothills on the north and the Thar Desert and Aravalli Mountains on the south. At the strategic choke point, just before reaching the fertile, well-watered Gangetic plain, sat Delhi.', metadata={'source': 'https://en.wikipedia.org/wiki/India', 'start_index': 123705}),
 Document(page_content='^ The northernmost point under Indian control is the disputed Siachen Glacier in Jammu and Kashmir; however, the Government of India regards the entire region of the former princely state of Jammu', metadata={'source': 'https://en.wikipedia.org/wiki/India', 'start_index': 93244}),
 Document(page_content='^ Fisher 2018, p.\xa083, "\xa0East of the lower Indus lay the in

#### to invoke a model we need to have the promopts and we can use the   langchain hub to get our pre defined promots

In [46]:
from langchain import hub
prompt = hub.pull("rlm/rag-prompt")

In [47]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

In [56]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

rag_chain = (
    {"context": retriver | format_docs , "question" : RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [61]:
for chuck in rag_chain.stream('where is Himalayas'):
  print(chuck, end="", flush=True)

Human: You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.
Question: where is Himalayas 
Context: Himalayas.[173] Immediately south of the emerging Himalayas, plate movement created a vast crescent-shaped trough that rapidly filled with river-borne sediment[174] and now constitutes the

^ The northernmost point under Indian control is the disputed Siachen Glacier in Jammu and Kashmir; however, the Government of India regards the entire region of the former princely state of Jammu

the Himalayan foothills on the north and the Thar Desert and Aravalli Mountains on the south. At the strategic choke point, just before reaching the fertile, well-watered Gangetic plain, sat Delhi.

^ Fisher 2018, p. 83, " East of the lower Indus lay the inhospitable Rann of Kutch and Thar Desert. East of the upper Indus lay t